# Unit 5 - Financial Planning


In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

ModuleNotFoundError: No module named 'alpaca_trade_api'

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
My_BTC = 1.2
My_ETH = 5.3

In [ ]:
# Crypto API URLs
BTC_URL = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
ETH_URL = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"
requests.get(BTC_URL)

In [ ]:
# Fetch current BTC price
Fetch_Data = requests.get(BTC_URL).json()
Price_BTC = Fetch_Data['data']['1']['quotes']['USD']['price']

# Fetch current ETH price
Fetch_Data = requests.get(ETH_URL).json()
Price_ETH = Fetch_Data['data']['1027']['quotes']['USD']['price']

# Compute current value of my crpto
My_BTC_Value = My_BTC * Price_BTC
My_ETH_Value = My_ETH * Price_ETH

# Print current crypto wallet balance
print(f"The current value of your {My_BTC} BTC is ${My_BTC_Value:0.2f}")
print(f"The current value of your {My_ETH} ETH is ${My_ETH_Value:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Current amount of shares
My_SPY = 50
My_AGG = 200

In [ ]:
# Set Alpaca API key and secret
Alpaca_API_Key = os.getenv("ALPACA_API_KEY")
Alpaca_Secret_Key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
Alpaca = tradeapi.REST(
    Alpaca_API_Key,
    Alpaca_Secret_Key,
    api_version="v2")

In [ ]:
# Format current date as ISO format
Now = pd.Timestamp("2020-10-27", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
Data = Alpaca.get_barset(
    tickers,
    timeframe,
    start = Now,
    end = Now
).df

# Preview DataFrame
Data

In [ ]:
# Pick AGG and SPY close prices
AGG_Close_Price = Data["AGG"]["close"][0]
SPY_Close_Price = Data["SPY"]["close"][0]

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${AGG_Close_Price}")
print(f"Current SPY closing price: ${SPY_Close_Price}")

In [ ]:
# Compute the current value of shares
My_SPY_Value = My_SPY * SPY_Close_Price
My_AGG_Value = My_AGG * AGG_Close_Price

# Print current value of share
print(f"The current value of your {My_SPY} SPY shares is ${My_SPY_Value:0.2f}")
print(f"The current value of your {My_AGG} AGG shares is ${My_AGG_Value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
Monthly_Income = 12000

# Create savings DataFrame
All_Savings = pd.DataFrame(
    [
        [My_BTC_Value + My_ETH_Value],
        [My_SPY_Value + My_AGG_Value]
    ],
    index = ["Cryptos Value", "Stocks Value"],
    columns = ["amount"]
)

# Display savings DataFrame
display(All_Savings)

In [ ]:
# Plot savings pie chart
All_Savings.plot(kind = "pie", y = "amount", figsize = (5,5))

In [ ]:
# Set ideal emergency fund
Emergency_Fund = Monthly_Income * 3

# Calculate total amount of savings
Total_Savings = All_Savings["amount"]["Cryptos Value"] + \
    All_Savings["amount"]["Stocks Value"]

# Validate saving health
if Total_Savings > Emergency_Fund:
    print("Congratulations for having enough money in this Fund!")
elif Total_Savings == Emergency_Fund:
    print("Congratulations for reaching your Financial Goals")
else:
    print(f"You are ${(Emergency_Fund - Total_Savings):0.2f} away from reaching your Goal")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2015-08-07', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-08-07', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
df_stock_data = Alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
mc = MCSimulation(
    df_stock_data,
    num_simulation = 500,
    num_trading_days = 30*252
)
mc

In [ ]:
# Printing the simulation input data
df_stock_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
mc.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
mc.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
mc.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
Summary_Statistics = mc.summarize_cumulative_return()

# Print summary statistics
print(Summary_Statistics)

### Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = Summary_Statistics[8] * initial_investment
ci_upper = Summary_Statistics[9] * initial_investment

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(Summary_Statistics[8] * initial_investment, 2)
ci_upper = round(Summary_Statistics[9] * initial_investment, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
mc_5Years = MCSimulation(
    df_stock_data,
    num_simulation = 500,
    num_trading_days = 5*252
)
mc_5Years

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
mc_5Years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
mc_5Years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
mc_5Years.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
Summary_Statistics_1 = mc_5Years.summarize_cumulative_return()

# Print summary statistics
print(Summary_Statistics)

In [ ]:
# Set initial investment
initial_investment_1 = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = round(Summary_Statistics_1[8] * initial_investment_1, 2)
ci_upper_five = round(Summary_Statistics_1[9] * initial_investment_1, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment_1} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
mc_10Years = MCSimulation(
    df_stock_data,
    num_simulation = 500,
    num_trading_days = 10*252
)
mc_10Years

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
mc_10Years.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
mc_10Years.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
mc_10Years.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
Summary_Statistics_2 = mc_10Years.summarize_cumulative_return()

# Print summary statistics
print(Summary_Statistics_2)

In [ ]:
# Set initial investment
initial_investment_2 = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = round(Summary_Statistics_2[8] * initial_investment_2, 2)
ci_upper_ten = round(Summary_Statistics_2[9] * initial_investment_2, 2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment_2} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")